# Summary

Created on Thu May 7 2020 <br>
@author: goutham <br>

## Multi Label Classifcation using keras Deep Neural Networks <br>

Topics
* Keras Neural Networks
* Multi Label Classification 
* Metric = 'accuracy'
* Model Evaluation using Custom Metric <br>

Packages Used: 
* keras
* numpy
* pandas <br>

The purpose of this code is to show how one can build a basic Multi Label Classification Neural Network using Keras.
This code also helps one understand what to look out for when using 'accuracy' as a metric for model evaluation in multi label classification problems with high target dimensionality and sparse data.



## Get the Data

In [0]:
# Input the Data
import pandas as pd
import numpy as np
dataGroup = pd.read_csv('YourPath/MLCSampleData.csv') # Insert your data path here

In [3]:
# Understand the Data
dataGroup.head()
# This is dummy data that I created to use for building a multi label classification keras neural network
# The label data is one-hot encoded - the way it needs to be fed into the output layer
# id column is a unique identifier for an instance 
# Feature0 to Feature99 are the 100 features that should be used to build the classifier
# Label0 to Label39 are the 40 distinct labels (output) available and the data is sparse

,id,Feature0,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,Feature11,Feature12,Feature13,Feature14,Feature15,Feature16,Feature17,Feature18,Feature19,Feature20,Feature21,Feature22,Feature23,Feature24,Feature25,Feature26,Feature27,Feature28,Feature29,Feature30,Feature31,Feature32,Feature33,Feature34,Feature35,Feature36,Feature37,Feature38,...,Label0,Label1,Label2,Label3,Label4,Label5,Label6,Label7,Label8,Label9,Label10,Label11,Label12,Label13,Label14,Label15,Label16,Label17,Label18,Label19,Label20,Label21,Label22,Label23,Label24,Label25,Label26,Label27,Label28,Label29,Label30,Label31,Label32,Label33,Label34,Label35,Label36,Label37,Label38,Label39
0,1,0.015835,-0.045127,0.094640,0.048653,-0.066024,-0.023271,-0.016250,0.033512,0.049166,-0.013469,0.022523,-0.079664,0.012362,-0.005313,0.016847,-0.007048,-0.014908,0.021623,0.071576,-0.024167,0.017272,-0.052168,-0.050011,-0.044022,-0.032369,0.028092,0.025928,0.038884,-0.050378,-0.056456,0.040243,-0.008851,0.042443,0.053557,-0.019753,0.049580,0.025410,-0.001016,-0.024046,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2,-0.063157,-0.017947,-0.030838,-0.007913,-0.028523,-0.001570,-0.002896,0.037894,0.095409,0.044269,-0.010836,-0.000652,-0.004343,-0.032067,-0.015429,0.033184,-0.020228,-0.005302,0.051265,0.037860,-0.047716,-0.017397,0.054960,-0.036262,0.021581,-0.057065,0.012647,-0.084299,0.008484,0.031245,-0.003681,0.058182,0.009452,-0.041775,0.007769,0.047030,0.036144,0.013338,0.029696,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,-0.037002,-0.030174,0.080122,-0.001855,-0.013632,-0.009644,-0.021854,-0.017841,-0.012354,-0.110699,-0.005091,-0.032056,0.028236,-0.040865,-0.016642,0.051011,-0.014348,-0.004535,0.069596,-0.018855,-0.029222,-0.061583,-0.028351,-0.038448,-0.056914,0.024536,0.052116,-0.072196,-0.044240,-0.045987,0.025957,-0.066354,0.038996,-0.042534,0.018681,0.072701,-0.055200,0.036336,-0.049919,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,4,-0.041543,-0.020182,0.002726,-0.080072,0.032774,-0.004889,0.036260,0.047002,0.003048,-0.036937,-0.066972,0.051171,0.047988,0.058928,-0.054135,0.013575,-0.017803,-0.008322,-0.040067,-0.023788,0.042533,-0.045643,0.016811,-0.039583,0.023409,-0.097692,0.013519,-0.036811,-0.006163,0.007077,0.059484,0.050211,0.025730,-0.071290,0.046276,0.032288,0.102312,0.016655,-0.021959,...,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,-0.041543,-0.020182,0.002726,-0.080072,0.032774,-0.004889,0.036260,0.047002,0.003048,-0.036937,-0.066972,0.051171,0.047988,0.058928,-0.054135,0.013575,-0.017803,-0.008322,-0.040067,-0.023788,0.042533,-0.045643,0.016811,-0.039583,0.023409,-0.097692,0.013519,-0.036811,-0.006163,0.007077,0.059484,0.050211,0.025730,-0.071290,0.046276,0.032288,0.102312,0.016655,-0.021959,...,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Train Test Split : Random Split 90:10 # Usually one does 80:20
X = dataGroup.iloc[:,1:101].copy() # get Feature0 to Feature99
y = dataGroup.iloc[:,101:].copy() # get Label0 to Label39
from sklearn.model_selection import train_test_split # random sampling split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0) # using random_state=0 for replication of results # should not use it when deploying

# transform the (Y) target into a numy array
y_train_list = y_train.values.tolist()
y_train = np.array(y_train_list)

y_test_list = y_test.values.tolist()
y_test = np.array(y_test_list)

## Build the Classifier

A very simple neural network was built as the primary objective was to focus on how the output layer and the loss function differs in a multi label classification from that of a conventional multi class classification network. <br>
This could be used as a base to build complex multi label classifiers.

In [5]:
# use keras framework for deep learning
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [6]:
# load a sequential model
model = Sequential()

# construct the network
model.add(Dense(units=64, activation='relu', input_dim=len(x_train.columns))) # len(x_train.columns) will automatically get the required count of nodes for input layer
model.add(Dense(units=128, activation='relu'))
model.add(Dense(len(y_train[0]), activation='sigmoid')) # Sigmoid over Softmax although we have multiple classes for independence across labels
                                                        # len(y_train[0]) = number of labels = number of nodes in output layer

# compile the model # using binary crossentropy over categorical crossentropy for independence across labels
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
# I'm using metrics=['accuracy']  in model.compile just to show what happens in model evaluation step later
# If the number of distinct labels is high and the data is sparse, i would ideally compile the model using the below line
#model.compile(loss='binary_crossentropy', optimizer='adam') 

# y_train is a Numpy arrays 
model.fit(x_train, y_train, epochs=150, batch_size=36, verbose = 0) # Using verbose = 0 as the interim output will make the notebook huge

Let's see what the model evaluation tells us

In [7]:
# Evaluate the model on the test data using 'evaluate'
print("Output when model evaluation was performed using model.evaluate() and ")
print("model compiled using <model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])>")
print()
results = model.evaluate(x_test, y_test, verbose = 0)
print("Test Accuracy","{:.2%}".format(results[1]))

Output when model evaluation was performed using model.evaluate() and 
model compiled using <model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])>

Test Accuracy 97.22%


Let's see how many records were accurately labelled by the model - our own evaluation metric

In [8]:
# Predict on test data using the model
labelsPredict = model.predict(x_test)

# I'm going to call a label 1 if its value > 0.5 and so, im going to use round function to get labels
# Threshold could be set dynamically for better performance of the model
labelsPredict = labelsPredict.round()

# Create Multi Label Vectors for the observed data # look at the output below to understand what this transformation does 
observed = []
for i in range(len(y_test)):
  multiLabel = ''
  for j in range(len(y_test[i,:])):
    multiLabel = multiLabel+str(y_test[i,j])
  observed.append(multiLabel)
print('Observed Data - first 5 records')
print(observed[:5])
print()

# Create Multi Label Vectors for the data predicted using model 1 # look at the output below to understand what this transformation does 
predicted = []
for i in range(len(labelsPredict)):
  multiLabel = ''
  for j in range(len(labelsPredict[i])):
    multiLabel = multiLabel+str(int(labelsPredict[i,j]))
  predicted.append(multiLabel)
print('Predicted Data - first 5 records')
print(predicted[:5])
print()

# For how many records did the model get all the labels correct?
is_accurate = []
for i in range(len(observed)):
  is_accurate.append(observed[i]==predicted[i])

CategoricalAccuracy = sum(is_accurate)/len(is_accurate) # (count of records accurately tagged)/(total number of records)
print('% of test records for which the model got all the labels correct is')
print("{:.2%}".format(CategoricalAccuracy))  
print()
print("total number of test records:",len(is_accurate))
print("accurately predicted records:",sum(is_accurate))


Observed Data - first 5 records
['0000000000000000000000100000000000000000', '0000000001000000000000000000000000000000', '0000010000000000000000000000000000000000', '0000000000000000000000000000000000000100', '0000000000000000000000100000000000000000']

Predicted Data - first 5 records
['0000000000000100000000000000000000010000', '0000000000000000000000000000000000000000', '0000000000000000000000000000000000000000', '0000000000000000000001000000000000000000', '0000000000000000000000100000000000000000']

% of test records for which the model got all the labels correct is
30.16%

total number of test records: 63
accurately predicted records: 19


As we can see, the code in the above cell calls the predictions for a record accurate only if the model gets all the labels correct for that record. But, the keras default accuracy metric considers each prediction as an individual event and so the discrepancy.

# Conclusion
There is **nothing wrong** with the keras accuracy metric. <br>
It is working the way it is intended to work. It considers each record-label prediction as an individual event.<br>
I've written a story on Medium explaining how 'Accuracy' works for a multi label classification problem and how to avoid any pitfalls that may arise from misinterpreting it.
Please refer to it for a non technical explanation on why the model evaluation is giving a test accuracy of 97% when the model only got multi-labels correct for 30% of the data.